# Plotting prelim Tracking using image features for lab meeting nov 22

Testing the Elliptical Fourier Descriptors (EFD) similarity between instances of cells

In [1]:
import dask.array as da
import napari
import btrack
from tqdm.auto import tqdm
import skimage
from skimage.io import imshow,  imsave
import matplotlib.pyplot as plt
import numpy as np
from pyefd import elliptic_fourier_descriptors
import seaborn as sns
import pandas as pd
plt.rcParams["figure.figsize"] = (7,7)

import cv2
def ffc(img, arg = 1100):
    if type(img) == da.core.Array:
        img = img.compute()
    # http://whitewell.sakura.ne.jp/OpenCV/py_tutorials/py_imgproc/py_filtering/py_filtering.html
    dst = cv2.blur(img, (arg, arg)) 
    #The original image/Image after Mean filter × Average brightness
    avg_hist = img.mean()
    # perform ffc
    img = (img/dst)*avg_hist
    ## convert back to dtype uint16
    img = img.astype('u2')
    
    return img

def ffc_stack(img_stack):
    stack = []
    for img in tqdm(img_stack):
        img = ffc(img)
        stack.append(img)
    img_stack = da.stack(stack, axis = 0)
    return img_stack

In [2]:
import napari

In [3]:
test_image = np.zeros((1000,1000), dtype = 'u1')

In [7]:
v = napari.Viewer()
# v.add_image(test_image)

/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [5]:
napari.__version__

'0.4.17'

In [6]:
stop

NameError: name 'stop' is not defined

### Load images from zarr and compress into max projections

In [ ]:
gfp = da.from_zarr('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch1.zarr/')
rfp = da.from_zarr('/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch2.zarr/')
gfp = da.sum(gfp, axis = 1, dtype='u2')
rfp = da.sum(rfp, axis = 1, dtype='u2')
masks = da.from_zarr(f'/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tzxy_stack_ch99.zarr/',)# chunks = gfp.chunks)

In [ ]:
gfp

In [ ]:
help(btrack.dataio.HDF5FileHandler)

### Apply a flatfield correction to the intensity images

This will be implemented in the tiling stage in the future

In [ ]:
gfp_c = ffc_stack(gfp)

In [ ]:
rfp_c = ffc_stack(rfp)

In [ ]:
gfp

In [ ]:
v = napari.Viewer()
v.add_image(gfp_c.astype('u1'))
v.add_image(rfp_c.astype('u1'))

In [ ]:
viewer = napari.Viewer()
viewer.add_image(gfp_c.astype('u2'), name="gfp cleaned", 
                 #contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')
viewer.add_image(rfp_c.astype('u2'), name="rfp _cLEAned", 
                # contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')
viewer.add_labels(masks, name = 'masks')
viewer.add_tracks(visualise_tracks, name = 'tracks_filtered')
viewer.add_tracks(visualise_raw_tracks, name = 'tracks')

In [ ]:
hist, bin_edges = np.histogram(rfp_c[0].compute().astype('u1'), bins = 2**16)

In [ ]:
plt.plot(bin_edges[0:-1], hist)
# plt.xlim(300,800)

In [ ]:
imshow(im)

In [ ]:
imsave('test.jpg',im.astype('u1'))

#### Compress into multichannel image for btrack

In [ ]:
image = da.stack([gfp, rfp], axis = -1)
image

## Define features

In [ ]:
FEATURES = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "solidity",
  "mean_intensity",
    "intensity_image"
]

In [ ]:
objects = btrack.utils.segmentation_to_objects(
    masks, 
    image,
    properties = tuple(FEATURES),
    use_weighted_centroid = False, 
)

#### Checking featuers were measured

In [ ]:
objects[0]

In [ ]:
### finding the EFD
for obj in tqdm(objects):
    ### extract the intensity image (1ch only)
    glimpse = obj.properties['intensity_image'][...,0]
    ### pad the glimpse to ensure only one object is identifiable
    glimpse = np.pad(glimpse, pad_width = 1)
    ### find the contours (zero because only one object)
    contours = skimage.measure.find_contours(glimpse, fully_connected='high', level = 0.5)[0]
    ### get the efd
    efd = elliptic_fourier_descriptors(contours, order=100, normalize=True)
#     obj.properties = {'efd': efd}
    flatten_efd = efd.flatten()
    obj.properties = {'efd flat': flatten_efd}

## Remove intensity image from objects 

btrack does not handle 2d arrays in objects

In [ ]:
for obj in tqdm(objects):
    obj.properties.pop('intensity_image')

In [ ]:
objects[0]

## Save out objects if desired

In [ ]:
with btrack.dataio.HDF5FileHandler(
     '/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/objects.h5', 'w', obj_type='obj_type_1',
) as hdf:
    hdf.write_segmentation(masks)
    hdf.write_objects(objects)

## Track

Redefine features to include `efd flat` and the two different `mean_intensity` channels

In [ ]:
FEATURES = ['efd flat',
                     "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "solidity",
  "mean_intensity-0",
  "mean_intensity-1",
]

In [ ]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file(
        '/home/dayn/analysis/BayesianTracker/models/particle_config.json'
    )
    tracker.verbose = True

    tracker.max_search_radius = 400
    
    # use visual features to track
    tracker.features = FEATURES
    
    # append the objects to be tracked
    tracker.append(objects)

    # set the volume
    tracker.volume=((0, 6048), (0, 6048), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=25)
    tracker.track(tracking_updates =['visual', 'motion'], step_size=10)


    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(f'/mnt/DATA/sandbox/pierre_live_cell_data/outputs/Replication_IPSDM_GFP/zarr/tracks.h5', obj_type='obj_type_1')

#     # get the tracks in a format for napari visualization (optional)
    visualise_tracks, properties, graph = tracker.to_napari(ndim=2)
    
    tracks = tracker.tracks

In [ ]:
tracks[100]

In [ ]:
print(tracks[0].properties['area'])

In [ ]:
print(tracks[0].properties['area'])

In [ ]:
plt.hist([len(track) for track in tracks])

In [ ]:
len([len(track) for track in tracks if len(track) > 15])

In [ ]:
plt.hist([len(track) for track in tracks if len(track) > 15])

### Filter tracks

In [ ]:
tracks_filt = [track for track in tracks if len(track) > 15]
visualise_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks_filt, ndim = 2)

In [ ]:
visualise_raw_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(gfp, name="gfp", 
                 #contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')
viewer.add_image(rfp, name="rfp", 
                # contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')
viewer.add_labels(masks, name = 'masks')
viewer.add_tracks(visualise_tracks, name = 'tracks_filtered')
viewer.add_tracks(visualise_raw_tracks, name = 'tracks')


### Prime tracks

(Greater than 50 frames length)

In [ ]:
prime_tracks = [track for track in tracks if len(track) > 50]

In [ ]:
track = prime_tracks[0]

In [ ]:
track

In [ ]:
for track in prime_tracks:
    plt.plot(track.properties['mean_intensity-1'], label = track.ID)
plt.legend()

In [ ]:
prime_tracks = [track for track in tracks if len(track) > 45]

In [ ]:
dfs = []
for track in prime_tracks:
    x = track.t
    y = track.properties['mean_intensity-1']
    d = {f'Mtb cell {track.ID}':y}
    df = pd.DataFrame(data=d, index = x)
    dfs.append(df)
df = pd.concat(dfs)
ax = sns.lineplot(data = df)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
dfs = []
for track in prime_tracks:
    if track.ID != 6677:
        continue
    x = track.t
    y = track.properties['mean_intensity-1']
    d = {f'Mtb cell {track.ID}':y}
    df = pd.DataFrame(data=d, index = x)
    dfs.append(df)
df = pd.concat(dfs)
ax = sns.lineplot(data = df)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

In [ ]:
track = [track for track in tracks if track.ID == 4356][0]

In [ ]:
track

# Why am i getting so many false positive dummy objects?

In [ ]:
objects

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 502, in mousePressEvent
    self._vispy_mouse_press(
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/base.py", line 184, in _vispy_mouse_press
    ev = self._vispy_canvas.events.mouse_press(**kwargs)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/_vispy/canvas.py", line 96, in _process_mouse_event
    super()._process_mouse_event(event)
  File "/home